<a href="https://colab.research.google.com/github/lukekolbe/AL-in-CreditScoring/blob/main/AL_CV_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
!conda install git pip

#import git
!pip install git+git://github.com/NUAA-AL/alipy.git

In [ ]:
### FOR RUNNING ON COLAB:
#update scikit-learn and imbalanced-learn to recent version
#!pip install sklearn-pandas-transformers
#!pip uninstall sklearn-pandas-transformers -y
#!pip uninstall sklearn_transformers_pandas -y


#!pip uninstall scikit-learn -y
#!pip install -U scikit-learn==1.0.1 #specific version, because skopt does not work with sklearn 1.0.0 

#pip uninstall imbalanced-learn -y
!pip install -U imbalanced-learn

!pip install scikit-optimize

!pip install git+https://github.com/NUAA-AL/alipy.git

#!pip install git+https://github.com/jundongl/scikit-feature
#!pip install git+https://github.com/lukekolbe/scikit-feature

#!pip install cvxpy

  Cloning git://github.com/lukekolbe/scikit-feature to c:\users\kolbeluk1\appdata\local\temp\9\pip-req-build-cn_ql178


  Running command git clone -q git://github.com/lukekolbe/scikit-feature 'C:\Users\kolbeluk1\AppData\Local\Temp\9\pip-req-build-cn_ql178'


In [ ]:
############ LIBRARIES

import os
import time
import datetime
import random
import multiprocessing
import pickle
import re
import copy
import gc
import sys
import json
gc.enable()

import warnings
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

'''import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
plt.style.use('default')
%matplotlib inline
import seaborn as sns'''

from itertools import cycle


import scipy.stats

#from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
#from sklearn.model_selection import StratifiedKFold  ##### what is this used for?
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score

# FOR CROSS VALIDATED HYPERPARAMETER TUNING
# use imblearn pipeline instead of sklearn pipeline to skip AL sampling process in the prediction phase
from imblearn.pipeline import Pipeline
from imblearn import FunctionSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression

from alipy import ToolBox
from alipy import query_strategy
from alipy.index import IndexCollection
from alipy import data_manipulate
import cvxpy

############ RANDOMNESS
# seed function
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 30
seed_everything(seed)


In [ ]:
os.getcwd()

'C:\\Users\\kolbeluk1\\AL_THESIS'

In [ ]:
#os.chdir('/gdrive/My Drive/ACTIVE LEARNING THESIS/')

# CV helper functions

In [ ]:
# get strategy

def strategy_getter(X, y, strategy_name="QueryInstanceRandom", train_idx = None, **kwargs):
    """Return the query strategy object from alipy package"""
    
    try:
        exec("from alipy.query_strategy import " + strategy_name)
    except:
        raise KeyError("Strategy " + strategy_name + " is not implemented in ALiPy.")
    strategy = None
    
    if train_idx is not None:
      strategy = eval(strategy_name + "(X=X, y=y, train_idx = train_idx, **kwargs)")
    else:
      strategy = eval(strategy_name + "(X=X, y=y, **kwargs)")
          
    # print(strategy)
    return strategy

In [ ]:
'''def pre_sampler(X=None
              , y=None):
  
  sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=seed) # THESE ARE POSITIONS, NOT INDICES    
  for label, unlabel in sss.split(X=X, y=y):
    label_idx, unlabel_idx = np.asarray(label), np.asarray(unlabel)    

  return X[label_idx,:], y[label_idx]'''

'def pre_sampler(X=None\n              , y=None):\n  \n  sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=seed) # THESE ARE POSITIONS, NOT INDICES    \n  for label, unlabel in sss.split(X=X, y=y):\n    label_idx, unlabel_idx = np.asarray(label), np.asarray(unlabel)    \n\n  return X[label_idx,:], y[label_idx]'

In [ ]:

class CustomScaler(BaseEstimator, TransformerMixin):
  
  def __init__(self, with_centering=True, with_scaling=True, seed=30):
        self.seed = seed
        self.with_centering = with_centering
        self.with_scaling = with_scaling
        self.scaler = RobustScaler(with_centering=self.with_centering, with_scaling = self.with_scaling)


  #estimator method
  def fit(self, X, y):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=int(0.5*len(X)), random_state=self.seed) # THESE ARE POSITIONS, NOT INDICES    
    for label, unlabel in sss.split(X=X, y=y):
      label_idx, unlabel_idx = np.asarray(label), np.asarray(unlabel)    
      #print(np.shape(y))
      #print(len(X[label_idx,:]))

      self.scaler.fit(X[label_idx,:])
      return self

  #transfprmation
  def transform(self, X, y = None):
    X = self.scaler.transform(X)
    return X



In [ ]:
# Custom Sampler Class, only run in the fit phase of the pipeline, skipped in predict phase
#####################################
def AL_resampler(X=None
              , y=None
              #, key # used to select correct strategy name from AL_Models
              , strategy_name = None
              , pass_index=False
              , model = None
              #, skip = False
              , seed=30
              , **kwargs):

  
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  
  # re-create the sample that was used to scale the data!
  # this is done so that the scaling is only done with data that is labeled, so that there is no leakage
  # this is the intial labeled sample that all strategies share
  sss = StratifiedShuffleSplit(n_splits=1, test_size=int(0.5*len(X)), random_state=seed) # THESE ARE POSITIONS, NOT INDICES    
  for label, unlabel in sss.split(X=X, y=y):
    label_idx, unlabel_idx = np.asarray(label), np.asarray(unlabel)    
        

  n_instances = int(0.5*len(unlabel_idx))

  '''#if skip and strategy_name != 'Oracle':
  if skip:
    # for models such as oracle, score, random, no AL parameters need tuning
    # Hence a random selection of instances is passed to the classifier for training
    # another small sample is drawn from the unlabeled data that is added to the labeled set
    # the size of the labeled data is then equivalent to the size of the sample that is curated by AL: 75% of the input data, 50% of which are picked randomly (and scaled with)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=n_instances, random_state=seed) # THESE ARE POSITIONS, NOT INDICES    
    for label, unlabel in sss.split(X=X[unlabel_idx], y=y[unlabel_idx]):
        label_add, unlabel_keep = np.asarray(label), np.asarray(unlabel)
    
    label_idx = np.concatenate((label_idx, unlabel_idx[label_add]))
    unlabel_idx = unlabel_idx[unlabel_keep]

  #else:'''
  ##############
  # initialize AL strategy
  ##############
  #t_idx = np.arange(len(X))
  t_idx = np.concatenate((label_idx, unlabel_idx))

  if pass_index and not strategy_name == 'QueryInstanceLAL':
    strategy = strategy_getter(X, y, strategy_name, t_idx, **kwargs)

  elif not pass_index and not strategy_name == 'QueryInstanceLAL':
    strategy = strategy_getter(X, y, strategy_name, **kwargs)

  # special case because of extra RF; might be prohibitively expensive to tune
  elif strategy_name == 'QueryInstanceLAL':
    #provide a path for the generic training data, downloaded from https://github.com/ksenia-konyushkova/LAL/tree/master/lal%20datasets
    #otherwise the data will be downloaded in every iteration, which is costly in a CV loop
    #data_path = '/gdrive/My Drive/ACTIVE LEARNING THESIS/LAL_trainingdata/'
    #file_path = '/gdrive/My Drive/ACTIVE LEARNING THESIS/LAL_trainingdata/LAL-iterativetree-simulatedunbalanced-big.npz'
    
    param_dict = {**kwargs}
    reg_est=param_dict.pop('reg_est', None)
    reg_depth=param_dict.pop('reg_depth', None)
    reg_feat=param_dict.pop('reg_feat', None)
    if reg_feat > np.shape(X)[1]:
      reg_feat = np.shape(X)[1]

    strategy = strategy_getter(X, y, strategy_name = strategy_name, mode='LAL_iterative', train_slt=False, **param_dict)
    starttime = time.time()
    strategy.download_data()
    print('duration of data download', starttime - time.time())
    strategy.train_selector_from_file(reg_est=reg_est, reg_depth=reg_depth, feat=reg_feat)

  ##############
  # run AL selection once
  ##############

  #fit model to then pass it as argument to some AL strategies
  if not model == None:
    model.fit(X[label_idx], y[label_idx])

  if strategy_name == 'QueryInstanceQUIRE': #strategy quire has no batch mode
    label_idx = IndexCollection(label_idx)
    unlabel_idx = IndexCollection(unlabel_idx)

    print('starting selection')
    for n in range(n_instances):
      selection = strategy.select(label_index=label_idx.index, unlabel_index=unlabel_idx.index)[0] # returns a list of len one, hence the [0]
      label_idx.update(selection)
      unlabel_idx.difference_update(selection)
      if n%10 == 0:
        print(f"selected {n} cases")
    label_idx = label_idx.index

  elif strategy_name in ['QueryInstanceBMDR', 'QueryInstanceSPAL']:
    select_idx = strategy.select(label_idx, unlabel_idx, model=model,batch_size=n_instances, qp_solver = 'OSQP')
    label_idx = np.concatenate((label_idx, select_idx))

  elif model == None:
    select_idx = strategy.select(IndexCollection(label_idx), IndexCollection(unlabel_idx), batch_size=n_instances)
    label_idx = np.concatenate((label_idx, select_idx))
  
  elif not model ==None:
    select_idx = strategy.select(label_idx, unlabel_idx, model=model, batch_size=n_instances)
    label_idx = np.concatenate((label_idx, select_idx))


  return X[label_idx,:], y[label_idx]

In [ ]:
 21%20

1

In [ ]:
'''from sklearn.ensemble import IsolationForest

def outlier_rejection(X, y, **kwargs):
    """This will be our function used to resample our dataset."""
    model = IsolationForest(max_samples = 'auto', contamination='auto', random_state=seed, n_jobs = -1, **kwargs)
    model.fit(X)
    o_pred = model.predict(X)
    print(np.mean(o_pred))
    return X[o_pred == 1], y[o_pred == 1], o_pred


reject_sampler = FunctionSampler(func=outlier_rejection)
X_inliers, y_inliers = reject_sampler.fit_resample(X_train, y_train)'''

'from sklearn.ensemble import IsolationForest\n\ndef outlier_rejection(X, y, **kwargs):\n    """This will be our function used to resample our dataset."""\n    model = IsolationForest(max_samples = \'auto\', contamination=\'auto\', random_state=seed, n_jobs = -1, **kwargs)\n    model.fit(X)\n    o_pred = model.predict(X)\n    print(np.mean(o_pred))\n    return X[o_pred == 1], y[o_pred == 1], o_pred\n\n\nreject_sampler = FunctionSampler(func=outlier_rejection)\nX_inliers, y_inliers = reject_sampler.fit_resample(X_train, y_train)'

# Load Data

In [ ]:
############ DATA IMPORT

## available datasets
# bene1         # some 

    # OK gmsc          # shape:  (150000, 68)
    # OK uk            # very low bad rate, shape:  (30000, 51), y mean:  0.04
    # OK lendingclub   # loooow performance, X shape:  (41623, 114) y shape:  (41623,) y mean:  0.1331235134420873    
    
    # OK bene2         # some learning, shape:  (7190, 28) >> 3 folds NO WASTE
    
    # bene1_nobins  # shape:  (3123, 18) # 5 folds NO WASTE
        # hmeq          # not promising either, shape:  (5960, 20) # 5 folds NO WASTE
        # australian    # very small dataset shape:  (690, 42) >> 10 folds NO WASTE
        # german        # shape:  (1000, 61) >> 10 folds NO WASTE
        # thomas        # loooow performance shape:  (1225, 28) >> 10 folds NO WASTE

# pakdd         # shape:  (50000, 373), y mean:  0.26082

dataset = "pakdd"
# C:\\Users\\kolbeluk1\\AL_THESIS
df = pd.read_csv('C:\\Users\\kolbeluk1\\AL_THESIS\\prepared_data\\{}.csv'.format(dataset))
#df = pd.read_csv('/gdrive/My Drive/ACTIVE LEARNING THESIS/prepared_data/{}.csv'.format(dataset))
#df = pd.read_csv('/gdrive/My Drive/ACTIVE LEARNING THESIS/prepared_data/bene2.csv')

# remove NA
df = df.dropna()
df.reset_index(drop = True, inplace = True)

#print(df)
# extract label
df['BAD'][df['BAD']=='BAD']  = 1
df['BAD'][df['BAD']=='GOOD'] = 0
df['BAD'] = df['BAD'].astype('int')


y_temp = df['BAD']
del df['BAD']

#one hot encoding
df = pd.get_dummies(df)

#transform to numpy array >> same location for df and X
X = df.to_numpy()
y = y_temp.to_numpy()

print("X type: ", type(X), "X shape: ",X.shape,"y shape: ", y.shape, "y mean: ", np.mean(y))
print (id(X), id(df))
#print(y)

# append_record: helper function that adds best-parameter for every model to dict and saves it
filename = f'{dataset}_tuned-params'

def append_record(record):
    with open(f'{filename}', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)
filename

X type:  <class 'numpy.ndarray'> X shape:  (50000, 373) y shape:  (50000,) y mean:  0.26082
763698962576 763701202176


'pakdd_tuned-params'

# Split off Validation Set

implement scheme for splitting a validation set

In [ ]:
if len(y) > 25000:
    validation_size = 5000
    folds = 3
    sss = StratifiedShuffleSplit(n_splits=1, test_size=validation_size, random_state=seed) # THESE ARE POSITIONS, NOT INDICES    
    for model, validation in sss.split(X=X, y=y):
        model_idx, validation_idx = np.asarray(model), np.asarray(validation)    

    print(np.sum(model_idx), np.sum(validation_idx))
    X_val, y_val = X[validation_idx,:], y[validation_idx]
    #X_model, y_model = X[model_idx,:], y[model_idx]

elif len(y) < 2000:
    folds = 10
    X_val, y_val = X, y

else:
    folds = 3
    X_val, y_val = X, y
    
print(np.shape(X), np.shape(X_val), 'folds: ', folds)

1126541891 123433109
(50000, 373) (5000, 373) folds:  3


In [ ]:
#hmeq

if len(y) > 5000:
    validation_size = 3000
    folds = 3
    sss = StratifiedShuffleSplit(n_splits=1, test_size=validation_size, random_state=seed) # THESE ARE POSITIONS, NOT INDICES    
    for model, validation in sss.split(X=X, y=y):
        model_idx, validation_idx = np.asarray(model), np.asarray(validation)    

    print(np.sum(model_idx), np.sum(validation_idx))
    X_val, y_val = X[validation_idx,:], y[validation_idx]

8948729 8809091


# CV Run

In [ ]:
# use imblearn pipeline to skip AL sampling process in the prediction phase
CLF_pipe = Pipeline(steps=[#('outlier', FunctionSampler(func=outlier_rejection)),
                      ('scaler', CustomScaler(seed=seed)),
                      ('AL_sampler', FunctionSampler(func=AL_resampler)),    # this will trigger a call to __init__
                      ('clf', LogisticRegression(random_state=seed))])

#cv_parameters_random = {'random': [{'strategy_name': 'QueryInstanceRandom'}]}


In [ ]:
## RUN TUNING FOR CLASSIFIER FIRST!

for key in ['random']:

  param_grid={'clf__solver': ['liblinear'], 
              'clf__penalty': ['l1', 'l2'], 
              'clf__C': [0.001,0.01,0.1,1.,10.,100.], 
              'clf__max_iter': [50, 100, 250, 500], 
              'clf__tol': [0.0001, 0.001, 0.0001, 0.00001],               
              #'clf__class_weight':['balanced', None], 
              'AL_sampler__kw_args': [{'strategy_name': 'QueryInstanceRandom'}]} #for  x in [125, 250, 500, 600], 'outlier__kw_args': transform_parameters, 

  grid_search_clf = RandomizedSearchCV(CLF_pipe, param_distributions=param_grid, n_iter = 500, cv=10, n_jobs=20, verbose=5, error_score="raise", refit='roc_auc', scoring=['roc_auc'], random_state=seed)
  grid_search_clf.fit(X_val, y_val)

  #### PRINT RESULTS
  '''print("Best parameters set found on development set:", "\n")
  print(grid_search_clf.best_params_)
  print("Grid scores on development set:", "\n")
  means_roc = grid_search_clf.cv_results_['mean_test_roc_auc']
  stds_roc = grid_search_clf.cv_results_['std_test_roc_auc']
  for means_roc, stds_roc, params in zip(means_roc, stds_roc, grid_search_clf.cv_results_['params']):
      print("%0.3f (+/-%0.03f) for %r"
            % (means_roc, stds_roc * 2, params))

  print("\n", "Detailed classification report:", "\n")
  print("The model is trained on the full development set.", "\n")
  print("The scores are computed on the full evaluation set.", "\n")
  y_pred = grid_search_clf.predict(X_model)
  print(classification_report(y_model, y_pred))'''

  ### SAVE RESULTS
  AL_params = copy.deepcopy(grid_search_clf.best_params_['AL_sampler__kw_args'])   #[i] for i in grid_search_clf.best_params_['sampler__kw_args'] if i not in ['key', 'name', 'sample_size']]
  AL_params.pop('pass_index', None)
  AL_params.pop('model', None)
  AL_params.pop('skip', None)

  CLF_params = copy.deepcopy(grid_search_clf.best_params_)
  CLF_params.pop('AL_sampler__kw_args', None)
  CLF_params.pop('outlier__kw_args', None)

  for clf_key in list(CLF_params.keys()):
    new_key = re.sub(r'clf__', '', clf_key)
    CLF_params[new_key] = CLF_params.pop(clf_key)

  cv_param_dict = {f'{key}': {#'outlier_rf': outlier_params,
                 'AL': AL_params,
                 'CLF': CLF_params}}
  
  append_record(cv_param_dict)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits


In [ ]:
clf = LogisticRegression(random_state=seed)

clf_tuned_params = copy.deepcopy(grid_search_clf.best_params_)
clf_tuned_params.pop('AL_sampler__kw_args', None)

for clf_key in list(clf_tuned_params.keys()):
  new_key = re.sub(r'clf__', '', clf_key)
  clf_tuned_params[new_key] = clf_tuned_params.pop(clf_key)

clf.set_params(**clf_tuned_params)
print(str(clf))

AL_pipe = Pipeline(steps=[#('outlier', FunctionSampler(func=outlier_rejection)),
                      ('scaler', CustomScaler(seed=seed)),
                      ('AL_sampler', FunctionSampler(func=AL_resampler)),    # this will trigger a call to __init__
                      ('clf', clf)])

cv_parameters = {'unc': [{'strategy_name': 'QueryInstanceUncertainty', 'model': clf, 'measure': m} for m in ['entropy', 'least_confident', 'margin', 'distance_to_boundary']]
                ,'qbc': [{'strategy_name': 'QueryInstanceQBC', 'model': clf, 'method': 'query_by_bagging', 'disagreement': d} for d in ['vote_entropy', 'KL_divergence']]
                #,'eer': [{'strategy_name': 'QueryExpectedErrorReduction', 'model': clf}] #slow, even without tuning >> maybe don't tune at all? See what the other tuning cycles choose as clf parameters?
                ,'dw': [{'strategy_name': 'QueryInstanceDensityWeighted', 'model': clf, 'uncertainty_meansure': u , 'distance': d, 'beta': b} for u in ['least_confident', 'margin', 'entropy'] for d in ['cityblock', 'cosine', 'euclidean'] for b in [0.5, 1, 2]]
                ,'density':[{'strategy_name': 'QueryInstanceGraphDensity', 'pass_index': True, 'metric': m} for m in ['canberra', 'jaccard', 'cosine', 'hamming']]
                ,'cors' :  [{'strategy_name': 'QueryInstanceCoresetGreedy', 'pass_index': True, 'distance': d} for d in ['cityblock', 'cosine', 'euclidean']]
                ,'lal': [{'strategy_name': 'QueryInstanceLAL', 'cls_est': cls_est, 'reg_est': reg_est, 'reg_depth': reg_depth, 'reg_feat': reg_feat} for cls_est in [16,32,64] for reg_est in [32,64,128] for reg_depth in [5,10,20] for reg_feat in [5,6,7]] # SLOOOW to tune
                #,'quire': [{'strategy_name': 'QueryInstanceQUIRE','pass_index': True, 'kernel': 'rbf', 'lambda': l, 'gamma': g} for g in [0.1, 1, 10] for l in [0.1, 1, 10]] #SLOOOOOOOOW, 40 mins at njobs = 1, cv = 2, n_iter = 25 >> extremely low spec tuning
                #,'bmdr': [{'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': b, 'gamma': g, 'rho': r, 'gamma_ker': gk} for b in [100, 1000, 10000] for g in [0.01, 0.1, 1] for r in [0.1, 1, 10] for gk in [0.1, 1, 10]] # issues with the solver ECOS; super slowto tune
                #,'spal': [{'strategy_name': 'QueryInstanceSPAL', 'kernel': 'rbf', 'mu': mu, 'gamma': g, 'rho': r, 'lambda_init': li, 'lambda_pace': lp, 'gamma_ker': gk} for mu in [0.01, 0.1, 1] for g in [0.01, 0.1, 1] for r in [0.1, 1, 10] for li in [0.01, 0.1, 1] for lp in [0.001, 0.01, 0.1] for gk in [0.1, 1, 10]]                
                ,'spal': [{'strategy_name': 'QueryInstanceSPAL', 'kernel': 'rbf', 'mu': mu, 'gamma': g,'rho': r, 'lambda_init': li, 'lambda_pace': lp, } for mu in [0.01, 0.1, 1] for g in [0.01, 0.1, 1] for r in [0.1, 1, 10] for li in [0.01, 0.1, 1] for lp in [0.001, 0.01, 0.1]]                
                ,'bmdr': [{'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': b, 'gamma': g, 'rho': r} for b in [100, 1000, 10000] for g in [0.01, 0.1, 1] for r in [0.1, 1, 10]] # issues with the solver ECOS; super slowto tune
                }

LogisticRegression(C=0.1, max_iter=50, random_state=30, solver='liblinear',
                   tol=1e-05)


In [ ]:
#import time

start = time.time()
print(start)
#'unc', 'qbc', 'dw', 'cors', 'density', 'bmdr', 'spal', 'eer', 'lal'
#for key in ['spal', 'bmdr']: 
for key in cv_parameters.keys(): #eer, quire, lal, spal, bmdr
#for key in ['bmdr']:
  loop_start = time.time()
  #iter_start= time.time()
  param_grid={'AL_sampler__kw_args': cv_parameters[key]} #for  x in [125, 250, 500, 600], 'outlier__kw_args': transform_parameters, 
  
  grid_search_AL = RandomizedSearchCV(AL_pipe, param_distributions=param_grid, n_iter=150, cv=folds, n_jobs=18, verbose=5, error_score="raise", refit='roc_auc', scoring=['roc_auc'], random_state=seed)
  #grid_search_AL = GridSearchCV(AL_pipe, param_grid=param_grid, cv=folds, n_jobs=20, verbose=5, error_score="raise", refit='roc_auc', scoring=['roc_auc'], random_state=seed)
  grid_search_AL.fit(X_val, y_val)

  AL_params = copy.deepcopy(grid_search_AL.best_params_['AL_sampler__kw_args'])   #[i] for i in grid_search_AL.best_params_['sampler__kw_args'] if i not in ['key', 'name', 'sample_size']]
  AL_params.pop('pass_index', None)
  AL_params.pop('model', None)
  AL_params.pop('skip', None)

  CLF_params = copy.deepcopy(grid_search_AL.best_params_)
  CLF_params.pop('AL_sampler__kw_args', None)
  #CLF_params.pop('outlier__kw_args', None)

  for clf_key in list(CLF_params.keys()):
    new_key = re.sub(r'clf__', '', clf_key)
    CLF_params[new_key] = CLF_params.pop(clf_key)

  cv_param_dict = {f'{key}': {#'outlier_rf': outlier_params,
                 'AL': AL_params,
                 'CLF': CLF_params}}
  
  append_record(cv_param_dict)
  

  '''iter_end = time.time()
  print(f'time {key}:',iter_end - iter_start)'''
  print(f'{key} time:',(time.time() - loop_start)/3600)
print('total time:',(time.time() - start)/3600)

1637874726.6753042
Fitting 3 folds for each of 4 candidates, totalling 12 fits
unc time: 0.00022953901025984022
Fitting 3 folds for each of 2 candidates, totalling 6 fits
qbc time: 0.0004285077253977458
Fitting 3 folds for each of 27 candidates, totalling 81 fits
dw time: 0.0011868402030732897
Fitting 3 folds for each of 4 candidates, totalling 12 fits
density time: 0.015081219209565056
Fitting 3 folds for each of 3 candidates, totalling 9 fits
cors time: 0.009375002185503642
Fitting 3 folds for each of 81 candidates, totalling 243 fits
duration of data download 0.0
Building lal regression model from .\LAL-iterativetree-simulatedunbalanced-big.npz
Done!
Oob score =  0.190898747395976
lal time: 0.16766286995675828
Fitting 3 folds for each of 150 candidates, totalling 450 fits
spal time: 0.8833321718374888
Fitting 3 folds for each of 27 candidates, totalling 81 fits
bmdr time: 10.521747084061305
total time: 11.599044068190787


In [ ]:
#UK

start = time.time()
print(start)
#'unc', 'qbc', 'dw', 'cors', 'density', 'bmdr', 'spal', 'eer', 'lal'
#for key in ['spal', 'bmdr']: 
for key in cv_parameters.keys(): #eer, quire, lal, spal, bmdr
#for key in ['bmdr']:
  loop_start = time.time()
  #iter_start= time.time()
  param_grid={'AL_sampler__kw_args': cv_parameters[key]} #for  x in [125, 250, 500, 600], 'outlier__kw_args': transform_parameters, 
  
  grid_search_AL = RandomizedSearchCV(AL_pipe, param_distributions=param_grid, n_iter=150, cv=folds, n_jobs=-1, verbose=5, error_score="raise", refit='roc_auc', scoring=['roc_auc'], random_state=seed)
  #grid_search_AL = GridSearchCV(AL_pipe, param_grid=param_grid, cv=folds, n_jobs=20, verbose=5, error_score="raise", refit='roc_auc', scoring=['roc_auc'], random_state=seed)
  grid_search_AL.fit(X_val, y_val)

  AL_params = copy.deepcopy(grid_search_AL.best_params_['AL_sampler__kw_args'])   #[i] for i in grid_search_AL.best_params_['sampler__kw_args'] if i not in ['key', 'name', 'sample_size']]
  AL_params.pop('pass_index', None)
  AL_params.pop('model', None)
  AL_params.pop('skip', None)

  CLF_params = copy.deepcopy(grid_search_AL.best_params_)
  CLF_params.pop('AL_sampler__kw_args', None)
  #CLF_params.pop('outlier__kw_args', None)

  for clf_key in list(CLF_params.keys()):
    new_key = re.sub(r'clf__', '', clf_key)
    CLF_params[new_key] = CLF_params.pop(clf_key)

  cv_param_dict = {f'{key}': {#'outlier_rf': outlier_params,
                 'AL': AL_params,
                 'CLF': CLF_params}}
  
  append_record(cv_param_dict)
  

  '''iter_end = time.time()
  print(f'time {key}:',iter_end - iter_start)'''
  print(f'{key} time:',(time.time() - loop_start)/3600)
print('total time:',(time.time() - start)/3600)

1637422400.4788964
Fitting 3 folds for each of 4 candidates, totalling 12 fits
unc time: 8.786307440863715e-05
Fitting 3 folds for each of 2 candidates, totalling 6 fits
qbc time: 0.0002318920029534234
Fitting 3 folds for each of 27 candidates, totalling 81 fits
dw time: 0.0007946791251500448
Fitting 3 folds for each of 4 candidates, totalling 12 fits
density time: 0.01941726697815789
Fitting 3 folds for each of 3 candidates, totalling 9 fits
cors time: 0.003652158644464281
Fitting 3 folds for each of 81 candidates, totalling 243 fits
duration of data download 0.0
Building lal regression model from .\LAL-iterativetree-simulatedunbalanced-big.npz
Done!
Oob score =  0.07399072928952011
lal time: 0.1668249283896552
Fitting 3 folds for each of 150 candidates, totalling 450 fits
spal time: 29.830344127284157
Fitting 3 folds for each of 27 candidates, totalling 81 fits
bmdr time: 43.33098174055417
total time: 73.35233521223068


In [ ]:
#### PRINT RESULTS
print("Best parameters set found on development set:", "\n")
print(grid_search_AL.best_params_)
print("Grid scores on development set:", "\n")
means_roc = grid_search_AL.cv_results_['mean_test_roc_auc']
stds_roc = grid_search_AL.cv_results_['std_test_roc_auc']
for means_roc, stds_roc, params in zip(means_roc, stds_roc, grid_search_AL.cv_results_['params']):
  print("%0.3f (+/-%0.03f) for %r"
        % (means_roc, stds_roc * 2, params))

print("\n", "Detailed classification report:", "\n")
print("The model is trained on the full development set.", "\n")
print("The scores are computed on the full evaluation set.", "\n")
y_pred = grid_search_AL.predict(X_model)
print(classification_report(y_model, y_pred))

#outlier_params = copy.deepcopy(grid_search_AL.best_params_['outlier__kw_args'])

Best parameters set found on development set: 

{'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': 1000, 'gamma': 0.01, 'rho': 0.1}}
Grid scores on development set: 

0.722 (+/-0.122) for {'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': 100, 'gamma': 0.01, 'rho': 0.1}}
0.722 (+/-0.122) for {'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': 100, 'gamma': 0.01, 'rho': 1}}
0.722 (+/-0.122) for {'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': 100, 'gamma': 0.01, 'rho': 10}}
0.722 (+/-0.122) for {'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': 100, 'gamma': 0.1, 'rho': 0.1}}
0.722 (+/-0.122) for {'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel': 'rbf', 'beta': 100, 'gamma': 0.1, 'rho': 1}}
0.722 (+/-0.122) for {'AL_sampler__kw_args': {'strategy_name': 'QueryInstanceBMDR', 'kernel':

In [ ]:
with open(filename) as f:
    param_list = [json.loads(line) for line in f]

param_dict = {}
for i in range(len(param_list)):
  strategy_short = list(param_list[i].keys())[0]
  param_dict[strategy_short] = param_list[i][list(param_list[i].keys())[0]]

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

# MISC STUFF

In [ ]:
len(X_val)

2000

In [ ]:
xn, yn, outliers = outlier_rejection(X_val, y_val, n_estimators=100)

0.7508


In [ ]:
cs = CustomScaler(with_centering= True, with_scaling=True, seed=30)
cs.fit(X_val, y_val)

X_test = cs.transform(np.array(X_val))

print(X_val[0,:])
print('\n', X_test[0,:])

(1438,)
719
[     0   6437  10355      0      0  27980      6      1      1 225001
      3     42      0  37000      3      2      1      3     26     26
      4      1      1      1      0     93  37000   2583]

 [ 0.         -0.44991428  0.08335666  0.          0.          2.54933941
  0.          0.          1.         -0.23970915  0.          0.33333333
  0.         -0.41101521  0.          0.          0.         -0.375
 -0.5        -0.38095238 -0.16666667  0.          0.          0.
  0.          3.88235294 -0.50818029 -1.36208538]


In [ ]:
2.79800e+04

27980.0

In [ ]:
scaler = RobustScaler(with_centering=True, with_scaling=True)
scaler2 = RobustScaler(with_centering=True, with_scaling=True)

scaler.fit(X_val)
X_t = scaler.transform(np.array(X_val)) #scaled version of full dataset

scaler2.fit(X_t)
X_t2 = scaler2.transform(np.array(X_t)) #scaled version of full dataset


In [ ]:
'''from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
#X_cv_train, y_cv_splits = skf.split(X_cv, y_cv)
indices = []
for train_index, test_index in skf.split(X_val, y_val):
  #print("TRAIN:", train_index, "TEST:", test_index)
  indices.append((train_index, test_index))

for i in range(5):
  tr_idx = indices[i][0]
  ts_idx = indices[i][1]

print(len(tr_idx), len(ts_idx))'''

4000 1000


In [ ]:
np.shape(indices)

(5, 2)